In [1]:
# Import deepos
import deepops as dp

# Give your tensor a cool name.

In [2]:
# make Tensors
a = dp.Tensor([1.0]*100000, name = "leo")
b = dp.Tensor([2.0]*100000, name = "brad pitt")

In [3]:
print(a.name)
print(b.name)

leo
brad pitt


# Forward Operations.

In [4]:
# Tensor Operations on CPU
print("Add:", a+b, "Mul:", a*b)

Add: Tensor(  shape: (100000,), numpy: ([3. 3. 3. ... 3. 3. 3.], dtype=float32), device: cpu) Mul: Tensor(  shape: (100000,), numpy: ([2. 2. 2. ... 2. 2. 2.], dtype=float32), device: cpu)


In [5]:
# Default Device is cpu
a

Tensor( name: leo,  shape: (100000,), numpy: ([1. 1. 1. ... 1. 1. 1.], dtype=float32), device: cpu)

In [6]:
# throw these Tensors in our precious GPU
a.device('gpu') # i.e gpu:0
b.device('gpu')

Tensor( name: brad pitt,  shape: (100000,), numpy: (<pycuda._driver.DeviceAllocation object at 0x7fcd4c6ab710>, dtype=float32), device: gpu)

In [7]:
# add them on GPU
addition = a.add(b)

In [8]:
# print the Addition
addition

Tensor(  shape: (100000,), numpy: (<pycuda._driver.DeviceAllocation object at 0x7fcd3c044170>, dtype=float32), device: gpu)

In [9]:
# adds beautifully with itself!
a.add(a)

Tensor(  shape: (100000,), numpy: (<pycuda._driver.DeviceAllocation object at 0x7fcd3c099bc0>, dtype=float32), device: gpu)

In [10]:
print(a + a)

Tensor(  shape: (100000,), numpy: (<pycuda._driver.DeviceAllocation object at 0x7fcd3c044ad0>, dtype=float32), device: gpu)


In [11]:
print(a * b)

Tensor(  shape: (100000,), numpy: (<pycuda._driver.DeviceAllocation object at 0x7fcd3c044b70>, dtype=float32), device: gpu)


In [18]:
repr(a.cpu())

'Tensor(  shape: (100000,), numpy: ([1. 1. 1. ... 1. 1. 1.], dtype=float32), device: cpu)'

# BackPropogration is real now.

In [13]:
Tensor = dp.Tensor
a1 = Tensor([1.0, 3.0, 1.0])
b1 = Tensor([7.0, 3.0, 5.0])

a2 = Tensor([4.0, 3.0, 1.0])
a3 = Tensor([3.0, 3.0, 1.0])
a4 = Tensor([7.0, 1.0, 6.0])
b2 = Tensor([1.0, 21.0, 12.0])

c = a1 * b1 + a3
d = a2 * b2 + a4
out = c * d

In [14]:
# calculate gradients in style!!
out.backward()

1.0

In [15]:
# output gradients
out.grad

1.0

In [16]:
# Gradients
_ = [print(f"Gradients for {tensor} tensor is : {tensor.grad}") for tensor in [a1,a2,a3,a4,b1,b2,c,d]]

Gradients for Tensor(  shape: (3,), numpy: ([1. 3. 1.], dtype=float32), device: cpu) tensor is : [ 77. 192.  90.]
Gradients for Tensor(  shape: (3,), numpy: ([4. 3. 1.], dtype=float32), device: cpu) tensor is : [ 10. 252.  72.]
Gradients for Tensor(  shape: (3,), numpy: ([3. 3. 1.], dtype=float32), device: cpu) tensor is : [11. 64. 18.]
Gradients for Tensor(  shape: (3,), numpy: ([7. 1. 6.], dtype=float32), device: cpu) tensor is : [10. 12.  6.]
Gradients for Tensor(  shape: (3,), numpy: ([7. 3. 5.], dtype=float32), device: cpu) tensor is : [ 11. 192.  18.]
Gradients for Tensor(  shape: (3,), numpy: ([ 1. 21. 12.], dtype=float32), device: cpu) tensor is : [40. 36.  6.]
Gradients for Tensor(  shape: (3,), numpy: ([10. 12.  6.], dtype=float32), device: cpu) tensor is : [11. 64. 18.]
Gradients for Tensor(  shape: (3,), numpy: ([11. 64. 18.], dtype=float32), device: cpu) tensor is : [10. 12.  6.]


# Neural Network is REAL!!.

# Lets Train our Stupid Little Model on some Stupid Little data :P

In [17]:
import numpy as np
from sklearn import datasets

import deepops as dp
from deepops.model import Model

class StupidLittleModel(Model):
    def __init__(self):
        super().__init__()
        self.dense1 = dp.layers.Dense(4, 16, activation="relu", name="dense1")
        self.dense2 = dp.layers.Dense(16, 16, activation="relu", name="dense1")
        self.dense3 = dp.layers.Dense(16, 1, activation="sigmoid", name="dense2")

    def forward(self, x):
        x = self.dense1(x)
        x = self.dense2(x)
        x = self.dense3(x)
        return x


train_X, train_y = datasets.load_iris(return_X_y=True)
X, y = np.asarray(train_X[:100]), np.asarray(train_y[:100])
yi = np.argwhere(y <= 1)
y = np.reshape(y[yi], (-1))
X = np.reshape(X[yi], (y.shape[0], -1))
X = (X - X.min()) / (X.max() - X.min())
X, y = np.asarray(X, np.float32), np.asarray(y, np.float32)


def loss(y, ypred):
    # MSE
    _loss = [(yb - ypb) * (yb - ypb) for yb, ypb in zip(y, ypred)]
    return sum(_loss) * dp.Tensor(1 / len(yb))


sequential = StupidLittleModel()
batch_size = 20


for steps in range(1000):
    ri = np.random.permutation(X.shape[0])[:batch_size]
    xb = X[ri]
    yb = y[ri]
    xb = [list(map(dp.Tensor, x)) for x in xb]
    # forward
    y_pred_b = list(map(sequential.forward, xb))
    yb = [dp.Tensor(y) for y in yb]
    total_loss = loss(yb, y_pred_b)
    # backward
    sequential.init_backward()
    total_loss.backward()
    # mini optimizer
    learning_rate = 0.1
    for p in sequential.parameters():
        p._data -= learning_rate * p.grad
    if steps % 1 == 0:
        print(f"step {steps} loss {total_loss.data}")

step 0 loss 0.48186054825782776
step 1 loss 0.29423007369041443
step 2 loss 0.2337043136358261
step 3 loss 0.16823597252368927
step 4 loss 0.1794496327638626
step 5 loss 0.15764467418193817
step 6 loss 0.14568139612674713
step 7 loss 0.1508547067642212
step 8 loss 0.15446768701076508
step 9 loss 0.13757599890232086
step 10 loss 0.1331002116203308
step 11 loss 0.12326835840940475
step 12 loss 0.11941482126712799
step 13 loss 0.1102059856057167
step 14 loss 0.134320467710495
step 15 loss 0.12006167322397232
step 16 loss 0.1174803152680397
step 17 loss 0.10730060189962387
step 18 loss 0.09436262398958206
step 19 loss 0.10174205154180527
step 20 loss 0.10223479568958282
step 21 loss 0.10556267946958542
step 22 loss 0.10153289139270782
step 23 loss 0.08809555321931839
step 24 loss 0.10135357826948166
step 25 loss 0.09899930655956268
step 26 loss 0.0787389874458313
step 27 loss 0.07799404114484787
step 28 loss 0.07733466476202011
step 29 loss 0.08798769116401672
step 30 loss 0.06664631515741

step 241 loss 0.008275728672742844
step 242 loss 0.004847466014325619
step 243 loss 0.008311986923217773
step 244 loss 0.008607068099081516
step 245 loss 0.005021723452955484
step 246 loss 0.005260996520519257
step 247 loss 0.009359478950500488
step 248 loss 0.00547941867262125
step 249 loss 0.006907690316438675
step 250 loss 0.005300631280988455
step 251 loss 0.006272898055613041
step 252 loss 0.004466293379664421
step 253 loss 0.008529753424227238
step 254 loss 0.006640049163252115
step 255 loss 0.005097014829516411
step 256 loss 0.00406254967674613
step 257 loss 0.0050222971476614475
step 258 loss 0.009896256029605865
step 259 loss 0.004756319336593151
step 260 loss 0.0033140971790999174
step 261 loss 0.0058510019443929195
step 262 loss 0.003263678401708603
step 263 loss 0.007941587828099728
step 264 loss 0.0037235193885862827
step 265 loss 0.004746223334223032
step 266 loss 0.006233767606317997
step 267 loss 0.0051253014244139194
step 268 loss 0.0032064635306596756
step 269 loss 0.

step 473 loss 0.0028883132617920637
step 474 loss 0.0023484413977712393
step 475 loss 0.0017243943875655532
step 476 loss 0.003344489261507988
step 477 loss 0.002041997155174613
step 478 loss 0.003407067386433482
step 479 loss 0.002036064164713025
step 480 loss 0.003694357117637992
step 481 loss 0.0018295006593689322
step 482 loss 0.0035901423543691635
step 483 loss 0.00264605856500566
step 484 loss 0.0026865568943321705
step 485 loss 0.0011393893510103226
step 486 loss 0.0015593772986903787
step 487 loss 0.0020182584412395954
step 488 loss 0.002012305660173297
step 489 loss 0.004488702397793531
step 490 loss 0.0012824125587940216
step 491 loss 0.00353572447784245
step 492 loss 0.0011939507676288486
step 493 loss 0.0013915677554905415
step 494 loss 0.0011070610489696264
step 495 loss 0.0017902288818731904
step 496 loss 0.0024349496234208345
step 497 loss 0.0014325309311971068
step 498 loss 0.002332422649487853
step 499 loss 0.002455966081470251
step 500 loss 0.0011731934500858188
step 

step 703 loss 0.0012025512987747788
step 704 loss 0.0012929168296977878
step 705 loss 0.0016354777617380023
step 706 loss 0.0014324423391371965
step 707 loss 0.0009942324832081795
step 708 loss 0.0009897196432575583
step 709 loss 0.0028924604412168264
step 710 loss 0.0015552144031971693
step 711 loss 0.0010427499655634165
step 712 loss 0.0030470348428934813
step 713 loss 0.0009158325265161693
step 714 loss 0.0008029171149246395
step 715 loss 0.0023273031692951918
step 716 loss 0.0018045175820589066
step 717 loss 0.0014983590226620436
step 718 loss 0.0008336193859577179
step 719 loss 0.0012598865432664752
step 720 loss 0.001057985587976873
step 721 loss 0.0005290746921673417
step 722 loss 0.0017961105331778526
step 723 loss 0.0012908641947433352
step 724 loss 0.0027661530766636133
step 725 loss 0.0009474988910369575
step 726 loss 0.002121723024174571
step 727 loss 0.0014666341012343764
step 728 loss 0.0011013713665306568
step 729 loss 0.0010586486896499991
step 730 loss 0.00259872619062

step 933 loss 0.00099033466540277
step 934 loss 0.0005110401543788612
step 935 loss 0.000967922795098275
step 936 loss 0.0011489596217870712
step 937 loss 0.002355084288865328
step 938 loss 0.0011668786173686385
step 939 loss 0.0005797470803372562
step 940 loss 0.00139764288906008
step 941 loss 0.000688600295688957
step 942 loss 0.0007843392086215317
step 943 loss 0.0008766269311308861
step 944 loss 0.0006658518686890602
step 945 loss 0.0006179582560434937
step 946 loss 0.0021130056120455265
step 947 loss 0.002067610854282975
step 948 loss 0.0013243526918813586
step 949 loss 0.001180299324914813
step 950 loss 0.0010904271621257067
step 951 loss 0.001049092854373157
step 952 loss 0.0006422174046747386
step 953 loss 0.0011146697215735912
step 954 loss 0.0009790546027943492
step 955 loss 0.0020292389672249556
step 956 loss 0.0006011283840052783
step 957 loss 0.0011117904214188457
step 958 loss 0.0011747190728783607
step 959 loss 0.0006692494498565793
step 960 loss 0.002185995690524578
ste

# Yay Loss is decreasing...